In [2]:
from bs4 import BeautifulSoup
import requests
import csv
from itertools import zip_longest
import regex as re

def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [51]:
pip install regex

     |████████████████████████████████| 686kB 4.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


## Download Example Setlist from Serebii

In [8]:
url = "https://www.serebii.net/card/skyridge/"

response = requests.get(url)

html = response.text

soup = BeautifulSoup(html, "html.parser")

raw_rows = soup.find_all("td", class_="cen")
extracted_rows = []

for num, pic, name in grouper(raw_rows, 3):
    num = num.text.replace("Skyridge", "").strip()
    name = name.text.strip()
    extracted_rows.append({"set_num": num, "card_name": name})
    

## Download Setlist from Price Charting

In [82]:
def get_set_list(pokemon_set: str):
    url = f"https://www.pricecharting.com/console/pokemon-{pokemon_set}?sort=model-number"

    response = requests.get(url)

    html = response.text

    soup = BeautifulSoup(html, "html.parser")

    raw_rows = soup.find_all("tr", id=True)

    set_list = []
    
    for row in raw_rows:
        #print(row)
        # Need to iterate now for all rows:
        row_text = re.split(r"#|\$|\+", row.text.replace("\n", ""))

        pokemon = row_text[0].strip()

        # Check what type of card it is
        if "Booster" in pokemon:
            continue
        elif "Holo" in pokemon:
            pokemon, card_type = re.split(r"\s\[", pokemon)
        else:
            card_type = ""

        # Get card link
        print(row)
        soup_row = BeautifulSoup(row.text, "html.parser")
        print(soup_row.find_all(href=re.compile("https://www.pricecharting.com/game/pokemon-\S*")))    
#         except:
#             print("\nerror row:\n")
#             print(row)
        
        # Name some stuff nicely
        number = row_text[1]
        card_type = card_type.strip("]")
        ungraded_price = row_text[2]
        psa_9_price = row_text[3]
        psa_10_price = row_text[4]

        
        # Create a clean row of data
        set_list.append(
            {
                "pokemon": pokemon,
                "card_type": card_type,
                "card_set": pokemon_set,
                "card_set_id": number,
                "ungraded_price": ungraded_price,
                "psa_9_price": psa_9_price,
                "psa_10_price": psa_10_price
            }
        )
        break
    return set_list

In [83]:
get_set_list("skyridge")


<tr data-product="842665" id="product-842665">
<td class="image">
<div>
<a href="https://www.pricecharting.com/game/pokemon-skyridge/aerodactyl-1" title="842665">
<img class="photo" loading="lazy" src="https://commondatastorage.googleapis.com/images.pricecharting.com/6ffada14be0237ba26f35f26a595a49895868e2c56601780c6b91ff9c9515c66/60.jpg"/>
</a>
</div>
</td>
<td class="title" title="842665">
<a href="/game/pokemon-skyridge/aerodactyl-1">Aerodactyl #1</a>
</td>
<td class="price numeric used_price">
<span class="js-price">$16.57</span>
</td>
<td class="price numeric cib_price">
<span class="js-price">$30.07</span>
</td>
<td class="price numeric new_price">
<span class="js-price">$108.24</span>
</td>
<td class="add_to">
<ul class="add_to collection">
<li class="add_to collection">
<ul class="list">
<li class="list-item">
<a class="js-add-one-click" data-product-id="842665" href="#" rel="nofollow">
<span class="small-desktop-visible">+ Collection</span>
                                    

[{'pokemon': 'Aerodactyl',
  'card_type': '',
  'card_set': 'skyridge',
  'card_set_id': '1',
  'ungraded_price': '16.57',
  'psa_9_price': '30.07',
  'psa_10_price': '108.24'}]

In [84]:
def get_sales_per_card(card_url):
    return